My assignment "Segmenting and Clustering Neighborhoods in Toronto"
=================================================================

Part 1: Import dataset and clean
------

In [39]:
#Import libraries and load file: Postals code
#------------------------------
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_0fc2bc806def43ae8a798aa0556ecb5e = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='lOe7clgCxH_aSCr51_yP_USjM3TCnNhxZOEirRiEuwvt',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_0fc2bc806def43ae8a798aa0556ecb5e.get_object(Bucket='courseracapstone-donotdelete-pr-dvw99rtixx8dd5',Key='DataToronto.xlsx')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_excel(body)
df.head()


,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn


In [40]:
#Filter rows en new dataframe, excluying 'Not assigned' rows in Borough column
df_filter = df[df['Borough'] != 'Not assigned']
df_filter.head()

,Postal Code,Borough,Neighborhood
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae


In [41]:
#Print number rows
df_filter.shape

(103, 3)

Part 2: Obtein latitude and longitude
------

In [42]:
#Read file with data latitude and longitude
#------------------------------------------
body = client_0fc2bc806def43ae8a798aa0556ecb5e.get_object(Bucket='courseracapstone-donotdelete-pr-dvw99rtixx8dd5',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_latlon = pd.read_csv(body)
df_latlon.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [43]:
#Print number rows
df_latlon.shape

(103, 3)

In [44]:
# define the dataframe columns
column_names = ['Postal Code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
df_complete = pd.DataFrame(columns=column_names)
df_complete.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude


In [45]:
#Fill latitude and longitude columns
for index, row in df_filter.iterrows():
    Postalcode = row['Postal Code']
    Borough_name = row['Borough']
    Neighborhood_name = row['Neighborhood']
    
    #Find latitude and longitude in other dataset
    df_temp = df_latlon.loc[df_latlon.loc[:, 'Postal Code'] == Postalcode]
    Latitude  = df_temp.iloc[0,1] #df_temp['Latitude']
    Longitude = df_temp.iloc[0,2] #df_temp['Longitude']
    #print(Postalcode,Borough_name,Neighborhood_name,Latitude,Longitude,"\n")

    df_complete = df_complete.append({'Postal Code': Postalcode,
                                      'Borough' : Borough_name,
                                      'Neighborhood': Neighborhood_name,
                                      'Latitude': Latitude,
                                      'Longitude': Longitude}, ignore_index=True)


In [46]:
#View new dataset
df_complete.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


Part 3: Show Boroughs of Toronto
------

In [47]:
#Filter rows en new dataframe, only  Boroughs of 'Toronto'
df_Toronto = df_complete[df_complete.Borough.isin(['Central Toronto', 'Downtown Toronto', 'East Toronto','West Toronto'])]
df_Toronto.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [48]:
#Show the Map
#------------
#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

venues_map = folium.Map(location=[43.7001114, -79.4162979], zoom_start=15) 

for lat, lng, label in zip(df_Toronto.Latitude, df_Toronto.Longitude, df_Toronto.Borough):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map
